In [ ]:
# This document serve as a tutorial to show how to convert MXNet model to CoreMLModel 
# Target: image classification model
# Require Libraries: mxnet, numpy, onnx_coreml, coremltools, copy, PIL

#Reference Tutorial: 
# Convert to Onxx: https://mxnet.apache.org/api/python/docs/tutorials/deploy/export/onnx.html
# Adding scale layer and normalized image: https://github.com/onnx/onnx-coreml/issues/338

In [2]:
import mxnet as mx
import numpy as np
from mxnet.contrib import onnx as onnx_mxnet
from onnx_coreml import convert

/Users/tomriddle/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/tomriddle/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/tomriddle/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/tomriddle/miniconda3/lib/python3.7/site-packages/tensorflow/

In [10]:
# Assume that you already have the trained model,
# The trained model will be saved in two files: model.json and model_weight.params

# First, we need to convert MXNet model to Onnx model (open neural network exchange)

# Location of the model
sym = './trained_models/myresnet_cat-symbol.json'
params = './trained_models/myresnet_cat-0010.params'

# The shape of the image is: 224 x 224
input_shape = (1,3,224,224)
onnx_file = 'myresnet_cat.onnx'

#convert to onnx
converted_model_path = onnx_mxnet.export_model(sym, params, [input_shape], np.float32, onnx_file)

W0317 20:30:52.791011 4724682048 _op_translations.py:660] Pooling: ONNX currently doesn't support pooling_convention. This might lead to shape or accuracy issues. https://github.com/onnx/onnx/issues/549


In [11]:
# 
# Image scale for ImageNet
"""
IMAGE_NET_MEAN = [0.485, 0.456, 0.406]
IMAGE_NET_STD = [0.229, 0.224, 0.225]
"""
scale = 1.0 / (0.226 * 255.0)
red_scale = 1.0 / (0.229 * 255.0)
green_scale = 1.0 / (0.224 * 255.0)
blue_scale = 1.0 / (0.225 * 255.0)

args = dict(is_bgr=False, red_bias = -(0.485 * 255.0) * red_scale  , green_bias = -(0.456 * 255.0) * green_scale,
            blue_bias = -(0.406 * 255.0) * blue_scale)

In [14]:
classes = ["bengal_cat",
"munchkin_cat",
"persian_cat",
"siamese_cat",
"turkishangora_cat"]

ml_model = convert(model= onnx_file,
                   mode='classifier',
                   class_labels = classes,
                   minimum_ios_deployment_target='12',
                  preprocessing_args=args) # or minimum_ios_deployment_target = '13'

ml_name = 'myresnet_cat.mlmodel'
ml_model.save(ml_name)

1/175: Converting Node Type BatchNormalization
2/175: Converting Node Type Conv
3/175: Converting Node Type BatchNormalization
4/175: Converting Node Type Relu
5/175: Converting Node Type MaxPool
6/175: Converting Node Type BatchNormalization
7/175: Converting Node Type Relu
8/175: Converting Node Type Conv
9/175: Converting Node Type BatchNormalization
10/175: Converting Node Type Relu
11/175: Converting Node Type Conv
12/175: Converting Node Type BatchNormalization
13/175: Converting Node Type Relu
14/175: Converting Node Type Conv
15/175: Converting Node Type Conv
16/175: Converting Node Type Add
17/175: Converting Node Type BatchNormalization
18/175: Converting Node Type Relu
19/175: Converting Node Type Conv
20/175: Converting Node Type BatchNormalization
21/175: Converting Node Type Relu
22/175: Converting Node Type Conv
23/175: Converting Node Type BatchNormalization
24/175: Converting Node Type Relu
25/175: Converting Node Type Conv
26/175: Converting Node Type Add
27/175: Conv

In [15]:
ml_model

input {
  name: "data"
  shortDescription: "MultiArray of shape (1, 1, 3, 224, 224). The first and second dimensions correspond to sequence and batch size, respectively"
  type {
    multiArrayType {
      shape: 3
      shape: 224
      shape: 224
      dataType: FLOAT32
    }
  }
}
output {
  name: "resnetv21_dense1_fwd"
  type {
    dictionaryType {
      stringKeyType {
      }
    }
  }
}
output {
  name: "classLabel"
  type {
    stringType {
    }
  }
}
predictedFeatureName: "classLabel"
predictedProbabilitiesName: "resnetv21_dense1_fwd"
metadata {
  userDefined {
    key: "coremltoolsVersion"
    value: "3.3"
  }
}

In [40]:
# Because it is more convenient to have input as Image in iOS app,
# we then need to change the input type from MultiArray to Image
import coremltools
import coremltools.proto.FeatureTypes_pb2 as ft 

spec = coremltools.utils.load_spec("myresnet_cat.mlmodel")

input = spec.description.input[0]
input.shortDescription = "Image type with shape: 224 x 224 (RGB)"
input.type.imageType.colorSpace = ft.ImageFeatureType.RGB
input.type.imageType.height = 224 
input.type.imageType.width = 224



In [4]:
# Next, we need to add a scale layer to normalize image input into range [0..1]
import copy

# Get all layers of the model
layers = spec.neuralNetworkClassifier.layers

# We then make a deepcop the layers before delete it
layers_copy = copy.deepcopy(layers)
del layers[:]

# Now, scale_layer becomes the first layer of the network
scale_layer = layers.add()
scale_layer.name = 'scale_layer'
scale_layer.input.append('data')
scale_layer.output.append('input_scaled')

# Image scale for ImageNet
"""
IMAGE_NET_MEAN = [0.485, 0.456, 0.406]
IMAGE_NET_STD = [0.229, 0.224, 0.225]
"""
#scale = 1.0 / (0.226 * 255.0)
red_scale = 1.0 / (0.229 * 255.0)
green_scale = 1.0 / (0.224 * 255.0)
blue_scale = 1.0 / (0.225 * 255.0)

#args = dict(is_bgr=False, red_bias = -(0.485 * 255.0) * red_scale  , green_bias = -(0.456 * 255.0) * green_scale,
            #blue_bias = -(0.406 * 255.0) * blue_scale)

params = scale_layer.scale
params.scale.floatValue.extend([red_scale, green_scale, blue_scale]) 
params.shapeScale.extend([3,1,1]) # shape of the scale vector 

# now add back the rest of the layers
layers.extend(layers_copy)

# we need to change the input of the second layer to match the output of the scale_layer
layers[1].input[0] = 'input_scaled'

In [5]:
from PIL import Image
# Get the model for testing
mlmodel = coremltools.models.MLModel(spec)

# Confirm the model has input as Image datatype
print(mlmodel)

input {
  name: "data"
  shortDescription: "MultiArray of shape (1, 1, 3, 224, 224). The first and second dimensions correspond to sequence and batch size, respectively"
  type {
    imageType {
      width: 224
      height: 224
      colorSpace: RGB
    }
  }
}
output {
  name: "resnetv21_dense1_fwd"
  type {
    dictionaryType {
      stringKeyType {
      }
    }
  }
}
output {
  name: "classLabel"
  type {
    stringType {
    }
  }
}
predictedFeatureName: "classLabel"
predictedProbabilitiesName: "resnetv21_dense1_fwd"
metadata {
  userDefined {
    key: "coremltoolsVersion"
    value: "3.3"
  }
}



In [39]:
# Load an image to test your model
img_path = './munchkin_cat.jpg'
read_img = Image.open(img_path)
# Resize image to the shape of your input image
resize_img = read_img.resize((224, 224))
resize_img.show()
y = mlmodel.predict({'data': resize_img})
print('output along channel at [0,0]: ', y)

output along channel at [0,0]:  {'resnetv21_dense1_fwd': {'bengal_cat': 0.7060546875, 'munchkin_cat': 1.912109375, 'persian_cat': -0.260009765625, 'siamese_cat': -3.39453125, 'turkishangora_cat': 0.75732421875}, 'classLabel': 'munchkin_cat'}


In [ ]:
#Save the modified model
model_name = "myresnet_cat_image(input).mlmodel"
coremltools.utils.save_spec(spec, model_name)